In [3]:
import cv2
import numpy as np
from random import randint
side=512
n_div=16
dataset_size=5000
input_data=np.empty((dataset_size,side,side))
output_data=np.empty((dataset_size,n_div,n_div,5))
for k in range(dataset_size):
    for i in range(n_div):
        for j in range(n_div):
            output_data[k][i][j][0]=randint(j*side/n_div,(j+1)*side/n_div)
            output_data[k][i][j][1]=randint(i*side/n_div,(i+1)*side/n_div)
            output_data[k][i][j][2]=randint(0,side-(j+1)*side/n_div)
            output_data[k][i][j][3]=randint(0,side-(i+1)*side/n_div)
            output_data[k][i][j][4]=0
for i in range(dataset_size):
    print(i)
    n_line=randint(1,20)
    image=np.full((side,side),255)
    coords=np.random.randint(0,side,(n_line,4))
    for coord in coords:    
        image=cv2.line(image,(coord[0],coord[1]),(coord[2],coord[3]),0,np.random.randint(1,5))
        j=int(coord[0]//(side/n_div))
        k=int(coord[1]//(side/n_div))
        output_data[i][j][k][0]=coord[0]
        output_data[i][j][k][1]=coord[1]
        output_data[i][j][k][2]=coord[2]
        output_data[i][j][k][3]=coord[3]
        output_data[i][j][k][4]=100
        
    input_data[i]=image
del cv2
input_data=np.reshape(input_data,(input_data.shape+(1,)))
np.savez_compressed('training_data.npz',input_data=input_data,output_data=output_data)
del np
del input_data,output_data,i,j,k,dataset_size,n_div,image,coords

MemoryError: 

In [ ]:
import numpy as np
data=np.load('training_data.npz')
input_data=data['input_data']
output_data=data['output_data']
del data,np

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D
from tensorflow.keras import initializers
from keras.callbacks import ModelCheckpoint


In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),use_bias=True,activation='relu',padding='same',kernel_initializer=initializers.RandomNormal(stddev=0.01),input_shape=input_data.shape[1:]))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(16,(3,3),use_bias=True,padding='same',activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01)))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(32,(3,3),use_bias=True,padding='same',activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01)))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(16,(3,3),use_bias=True,padding='same',activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01)))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(8,(3,3),use_bias=True,padding='same',activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01)))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(5,(3,3),use_bias=True,padding='same',activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01)))
print(model.summary())

In [ ]:
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
checkpoint = ModelCheckpoint("model.hdf5", monitor='loss', verbose=1,save_best_only=True, mode='auto', period=1)
history=model.fit(input_data, output_data,
          batch_size=10,
          epochs=100,
          callbacks=[checkpoint],
          shuffle=True,
          validation_split=0.4 
                 )

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()